In [ ]:
import boto3
from botocore.exceptions import ClientError
def get_cross_account_client(service: str, account_id: str, role_name: str):
    """
    Creates a boto3 client for the specified service using cross-account role assumption.
    
    Args:
        service (str): AWS service name (e.g., 'logs', 'cloudwatch')
        account_id (str): AWS account ID to access
        role_name (str): IAM role name to assume
        
    Returns:
        boto3.client: Service client with assumed role credentials
    """
    try:
        # Create STS client
        sts_client = boto3.client('sts')
        
        # Define the role ARN
        role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"
        
        # Assume the role
        assumed_role = sts_client.assume_role(
            RoleArn=role_arn,
            RoleSessionName=f"CrossAccount{service.capitalize()}Session"
        )
        
        # Extract temporary credentials
        credentials = assumed_role['Credentials']
        
        # Create client with assumed role credentials
        client = boto3.client(
            service,
            aws_access_key_id=credentials['AccessKeyId'],
            aws_secret_access_key=credentials['SecretAccessKey'],
            aws_session_token=credentials['SessionToken']
        )
        
        print(f"Successfully created cross-account client for {service} in account {account_id}")
        return client
        
    except Exception as e:
        print(f"Error creating cross-account client for {service}: {e}")
        raise e

: 